In [1]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 12.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/My Drive/DL Project

Mounted at /content/gdrive/
/content/gdrive/My Drive/DL Project


In [3]:
import os
import requests
from huggingface_hub import hf_hub_download

# Create the data folder if it doesn't exist
data_folder = 'data'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# URLs for the files
vocab_url = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/tokenizer/vocab.json'
merges_url = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/tokenizer/merges.txt'
# ckpt_url = 'https://huggingface.co/runwayml/stable-diffusion-v1-5/raw/main/v1-5-pruned-emaonly.ckpt'

# Download vocab.json and merges.txt
vocab_path = os.path.join(data_folder, 'vocab.json')
merges_path = os.path.join(data_folder, 'merges.txt')

response_vocab = requests.get(vocab_url)

with open(vocab_path, 'wb') as f:
    f.write(response_vocab.content)

response_merges = requests.get(merges_url)
with open(merges_path, 'wb') as f:
    f.write(response_merges.content)

# # Download v1-5-pruned-emaonly.ckpt
# ckpt_path = os.path.join(data_folder, 'v1-5-pruned-emaonly.ckpt')
# response_ckpt = requests.get(ckpt_url)
# with open(ckpt_path, 'wb') as f:
#     f.write(response_ckpt.content)

print("Files downloaded and saved to the 'data' folder.")

repo_id = "runwayml/stable-diffusion-v1-5"
filename = "v1-5-pruned-emaonly.ckpt"

# Download the model file
ckpt_path = hf_hub_download(repo_id=repo_id, filename=filename, cache_dir=data_folder)

print(f"Model downloaded and saved to {ckpt_path}")

Files downloaded and saved to the 'data' folder.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model downloaded and saved to data/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9/v1-5-pruned-emaonly.ckpt


In [4]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch
import importlib

importlib.reload(model_loader)
importlib.reload(pipeline)

DEVICE = "cpu"

ALLOW_CUDA = True
ALLOW_MPS = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("./data/vocab.json", merges_file="./data/merges.txt")
model_file = "./data/models--runwayml--stable-diffusion-v1-5/snapshots/1d0c4ebf6ff58a5caecab40fa1406526bca4b5b9/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## TEXT TO IMAGE

#

Using device: cuda


In [5]:
import latent_space_manipulator
import cross_attention_map
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)
importlib.reload(pipeline)
importlib.reload(cross_attention_map)

<module 'cross_attention_map' from '/content/gdrive/MyDrive/DL Project/cross_attention_map.py'>

#Automobile

In [7]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus in city"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000042070.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000042070-city.jpg")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Subject Info: {'bus': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:23,  1.66it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


/content/gdrive/MyDrive/DL Project/subject_attention_maps.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cumulative_attention_maps[subject] = tensor(cumulative_map)
  8%|▊         | 3/40 [00:02<00:26,  1.38it/s]

...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:23<00:00,  1.65it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.68it/s]

dict_keys([220, 200, 240, 780, 999])


In [8]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus on highway"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000042070.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000042070-highway.jpg")

Subject Info: {'bus': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.06it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:24,  1.52it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


In [10]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="car in city"
focus="car"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000097679.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/0000000097679-city.jpg")

Subject Info: {'car': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.61it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:22,  1.63it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:21<00:00,  1.71it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.79it/s]

dict_keys([220, 200, 240, 780, 999])


In [11]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="car on highway"
focus="car"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000097679.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/0000000097679-highway.jpg")

Subject Info: {'car': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.80it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.54it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:23<00:00,  1.68it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.69it/s]

dict_keys([220, 200, 240, 780, 999])


In [12]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus in city"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000122606.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/0000000122606-city.jpg")

Subject Info: {'bus': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.19it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.59it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:21<00:00,  1.75it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.78it/s]

dict_keys([220, 200, 240, 780, 999])


In [13]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus on highway"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000122606.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/0000000122606-highway.jpg")

Subject Info: {'bus': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.19it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.59it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:21<00:00,  1.71it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.77it/s]

dict_keys([220, 200, 240, 780, 999])


In [14]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus in city"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000165039.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000165039-city.jpg")

Subject Info: {'bus': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.05it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.56it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.70it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.74it/s]

dict_keys([220, 200, 240, 780, 999])


In [15]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus on highway"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000165039.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000165039-highway.jpg")

Subject Info: {'bus': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.18it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.58it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


In [17]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle in city"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000165518.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000165518-city.jpg")

Subject Info: {'motorcycle': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.34it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.60it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.65it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


In [18]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle on highway"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000165518.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000165518-highway.jpg")

Subject Info: {'motorcycle': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.07it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:24,  1.52it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


In [19]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle in city"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000230008.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000230008-city.jpg")

Subject Info: {'motorcycle': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.14it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.60it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:21<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.79it/s]

dict_keys([220, 200, 240, 780, 999])


In [20]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle on highway"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000230008.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000230008-highway.jpg")

Subject Info: {'motorcycle': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.25it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.59it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.71it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


In [21]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus in city"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000296224.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000296224-city.jpg")

Subject Info: {'bus': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.11it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.56it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.70it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:23<00:00,  1.74it/s]

dict_keys([220, 200, 240, 780, 999])


In [22]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus on highway"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000296224.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000296224-highway.jpg")

Subject Info: {'bus': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.36it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.56it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.75it/s]

dict_keys([220, 200, 240, 780, 999])


In [23]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus in city"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000460347.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8
## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000460347-city.jpg")

Subject Info: {'bus': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.05it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:22,  1.62it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.77it/s]

dict_keys([220, 200, 240, 780, 999])


In [24]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="bus on highway"
focus="bus"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000460347.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000460347-highway.jpg")

Subject Info: {'bus': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.04it/s]

dict_keys([220, 200, 240, 780, 999])


  5%|▌         | 2/40 [00:00<00:17,  2.22it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.71it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


In [25]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle in city"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000480985.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000480985-city.jpg")

Subject Info: {'motorcycle': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.16it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:24,  1.50it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


In [26]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="motorcycle on highway"
focus="motorcycle"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000480985.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000480985-highway.jpg")

Subject Info: {'motorcycle': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.16it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.57it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.71it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.75it/s]

dict_keys([220, 200, 240, 780, 999])


In [27]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="car in city"
focus="car"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000555050.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000555050-city.jpg")

Subject Info: {'car': 0, 'city': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.10it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.57it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])


In [28]:
importlib.reload(model_loader)

importlib.reload(latent_space_manipulator)

importlib.reload(cross_attention_map)
importlib.reload(pipeline)

# prompt = "A dog sitting"
# prompt = "human News anchor is delivering the weather report."
prompt="car on highway"
focus="car"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 9  # min: 1, max: 14


## (Iprompt - Iuncondprompt)*cfg_scale + Iuncondprompt = I final
## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "static/Automobile/000000555050.jpg"

input_image = Image.open(image_path)
# input_image = None
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.8

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 40

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
    focus=focus
)

# Combine the input image and the output image into a single image.
img = Image.fromarray(output_image)
img.save("static/Automobile/000000555050-highway.jpg")

Subject Info: {'car': 0, 'highway': 2}
Timesteps for manipulating latent space: [tensor(220), tensor(200), tensor(240), tensor(780)]


  2%|▎         | 1/40 [00:00<00:04,  8.29it/s]

dict_keys([220, 200, 240, 780, 999])
dict_keys([220, 200, 240, 780, 999])


  8%|▊         | 3/40 [00:01<00:23,  1.57it/s]

dict_keys([220, 200, 240, 780, 999])


...

dict_keys([220, 200, 240, 780, 999])


 98%|█████████▊| 39/40 [00:22<00:00,  1.73it/s]

dict_keys([220, 200, 240, 780, 999])


100%|██████████| 40/40 [00:22<00:00,  1.76it/s]

dict_keys([220, 200, 240, 780, 999])
